In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

ModuleNotFoundError: No module named 'torchtext'

In [ ]:


# Load the dataset
df = pd.read_csv('../../../datasets/bbc-text.csv')  # Replace with your dataset path
print(df.head())

# Preprocess the data
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

# Split into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenizer and Vocabulary Building
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter['text']:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_df), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Custom Dataset Class
class BBCDataset(Dataset):
    def __init__(self, dataframe, vocab, tokenizer, max_length=500):
        self.dataframe = dataframe
        self.vocab = vocab
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text']
        label = self.dataframe.iloc[idx]['category_encoded']
        tokens = [self.vocab[token] for token in self.tokenizer(text)]
        if len(tokens) < self.max_length:
            tokens += [0] * (self.max_length - len(tokens))  # Padding
        else:
            tokens = tokens[:self.max_length]  # Truncating
        return torch.tensor(tokens, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# Prepare Dataloaders
train_dataset = BBCDataset(train_df, vocab, tokenizer)
test_dataset = BBCDataset(test_df, vocab, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the Model
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim, num_classes)
        self.pool = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):
        embedded = self.embedding(x)  # (batch_size, seq_length, embed_dim)
        embedded = embedded.permute(0, 2, 1)  # (batch_size, embed_dim, seq_length)
        pooled = self.pool(embedded).squeeze(2)  # (batch_size, embed_dim)
        output = self.fc(pooled)  # (batch_size, num_classes)
        return output



In [ ]:
# Model Parameters
vocab_size = len(vocab)
embed_dim = 100
num_classes = len(label_encoder.classes_)

model = TextClassificationModel(vocab_size, embed_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for texts, labels in train_loader:
        optimizer.zero_grad()a
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for texts, labels in test_loader:
        outputs = model(texts)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy:.4f}')
